In [1]:
import csv

In [4]:
def initializing(uploaded_file_path):
    with open(uploaded_file_path, 'r', encoding='utf-8') as txt_file, open("data.csv", 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Timestamp', 'Sender', 'Message'])

        timestamp = ""
        sender = ""
        message = ""

        for line in txt_file:
            parts = line.split(' - ', 1)
            if len(parts) == 2:
                timestamp, content = parts
                sender_end_index = content.find(': ')
                if sender_end_index != -1:
                    sender = content[:sender_end_index]
                    message = content[sender_end_index + 2:].strip()
                else:
                    sender = content.strip()
                    message = ""
            else:
                message += line.strip()

            if timestamp and sender and message:
                csv_writer.writerow([timestamp, sender, message])
                timestamp = sender = message = ""
        
        if timestamp and sender and message:
            csv_writer.writerow([timestamp, sender, message])

In [6]:
initializing('uj.txt')

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = (
    "data.csv"
)

loader = CSVLoader(file_path=file_path)
data = loader.load()

for record in data[:2]:
    print(record)

page_content='Timestamp: 8/29/23, 8:37 PM
Sender: Ujjwal Kala
Message: .' metadata={'source': 'data.csv', 'row': 0}
page_content='Timestamp: 8/29/23, 8:37 PM
Sender: Ujjwal Kala
Message: https://chat.whatsapp.com/COi6PxBqKAZ8Ql4W4ibkdh' metadata={'source': 'data.csv', 'row': 1}


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()
google_gemini_api=os.getenv("GOOGLE_API_KEY")
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm_model=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=google_gemini_api)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1723349130.566988   13231 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1723349130.573131   13231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723349130.578024   13231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723349130.578558   13231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
text=text_splitter.split_documents(data)

In [12]:
text[50].page_content

'Timestamp: 9/10/23, 6:23\u202fPM\nSender: anurag\nMessage: Dusre wale ka yahe aaya kya'

In [13]:
from langchain.vectorstores import Chroma
persist_directory="db"
vectordb=Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)

In [14]:
vectordb.persist()
vectordb=None
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [15]:
retriver=vectordb.as_retriever()

In [16]:
docs=retriver.get_relevant_documents("is ujjwal is good friend")

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [17]:
docs

[Document(metadata={'row': 212, 'source': 'data.csv'}, page_content='Timestamp: 11/5/23, 8:58\u202fPM\nSender: anurag\nMessage: ujjwal apna cs ka ppt bhej'),
 Document(metadata={'row': 79, 'source': 'data.csv'}, page_content='Timestamp: 9/11/23, 11:09\u202fPM\nSender: Ujjwal Kala\nMessage: yaar ye smj ni aya aaj wala'),
 Document(metadata={'row': 190, 'source': 'data.csv'}, page_content='Timestamp: 10/16/23, 11:06\u202fAM\nSender: Ujjwal Kala\nMessage: ok sir'),
 Document(metadata={'row': 240, 'source': 'data.csv'}, page_content='Timestamp: 1/7/24, 12:24\u202fPM\nSender: Ujjwal Kala\nMessage: bhai🥰😍 client ki spelling thik krle bas 😄 ab roz tu party dega,💥💥😎😎')]

In [18]:
from langchain.chains import RetrievalQA
llm_model=ChatGoogleGenerativeAI(model="gemini-1.5-pro",google_api_key=google_gemini_api)
qa_chain=RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever=retriver,
    return_source_documents=True
)

I0000 00:00:1723349538.790944   13231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723349538.791686   13231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [19]:
query="is ujjwal is good friend"
llm_response=qa_chain(query)
llm_response

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'is ujjwal is good friend',
 'result': 'I cannot answer this question. There is no information about the nature of their friendship. \n',
 'source_documents': [Document(metadata={'row': 212, 'source': 'data.csv'}, page_content='Timestamp: 11/5/23, 8:58\u202fPM\nSender: anurag\nMessage: ujjwal apna cs ka ppt bhej'),
  Document(metadata={'row': 79, 'source': 'data.csv'}, page_content='Timestamp: 9/11/23, 11:09\u202fPM\nSender: Ujjwal Kala\nMessage: yaar ye smj ni aya aaj wala'),
  Document(metadata={'row': 190, 'source': 'data.csv'}, page_content='Timestamp: 10/16/23, 11:06\u202fAM\nSender: Ujjwal Kala\nMessage: ok sir'),
  Document(metadata={'row': 240, 'source': 'data.csv'}, page_content='Timestamp: 1/7/24, 12:24\u202fPM\nSender: Ujjwal Kala\nMessage: bhai🥰😍 client ki spelling thik krle bas 😄 ab roz tu party dega,💥💥😎😎')]}